# Examine the dataset which chatgpt 4o mini selects

In [17]:
import torch
import pdb
model_a = '/data/nee7ne/effi_cot/saved_models/effi_cot/old_vanilla/sentence_transformer/model.pt'
model_b_dir = '/data/nee7ne/effi_cot/saved_models/effi_cot/vanilla/gsm8k/sentence_transformer_ckpts/'
def compare_model_parameters(model_a, model_b_dir):
    """Compares the parameters of two PyTorch models."""
    # Check if the models have the same parameters
    # Load the models
    model_a = torch.load(model_a)
    for epoch in range(15):
        model_b = model_b_dir + f"model_epoch_{epoch}.pt"
        model_b = torch.load(model_b)

        if set(model_a.keys()) != set(model_b.keys()):
            return False

        # Iterate through the parameters and compare them
        gap = 0
        for param_name in model_a.keys():
            param_a = model_a[param_name]
            param_b = model_b[param_name]

            # Check if the parameters are equal
            # print("model_a device", param_a.device)
            # print("model_b device", param_b.device)
            param_a = param_a.cpu()
            param_b = param_b.cpu()
            gap += torch.abs(param_a - param_b).sum()
        print(f"epoch {epoch} gap: {gap}")
            # if not torch.equal(param_a, param_b):
            #     # print where the parameters are not equal
            #     print(f"Parameters are not equal at {param_name}")
            #     print(f"Difference: {torch.abs(param_a - param_b).sum()}")
                # return False
    return True

compare_model_parameters(model_a, model_b_dir)

epoch 0 gap: 36908.96875
epoch 1 gap: 29977.525390625
epoch 2 gap: 25434.9296875
epoch 3 gap: 21600.6796875
epoch 4 gap: 18427.546875
epoch 5 gap: 15698.2431640625
epoch 6 gap: 13269.2626953125
epoch 7 gap: 11078.9365234375
epoch 8 gap: 9217.1162109375
epoch 9 gap: 7469.4755859375
epoch 10 gap: 5863.73291015625
epoch 11 gap: 4344.67333984375
epoch 12 gap: 2882.39306640625
epoch 13 gap: 1436.8564453125
epoch 14 gap: 0.6747385859489441


True

In [ ]:


def generate_reasoning(self, query: str) -> str:
        """
        Generate detailed reasoning for a given query.

        Args:
            query: The mathematical problem to solve

        Returns:
            Detailed step-by-step reasoning
        """
        messages = [
            {"role": "system", "content": "You are a math tutor who provides detailed step-by-step solutions to math problems."},
            {"role": "user", "content": f"Problem: {query}\n\nPlease solve this step-by-step, showing all your work."}
        ]

        try:
            response = requests.post(
                self.api_url,
                headers=self.headers,
                json={
                    "model": self.model,
                    "messages": messages,
                    "temperature": 0.2,
                }
            )
            response.raise_for_status()
            reasoning = response.json()["choices"][0]["message"]["content"]
            return reasoning
        except Exception as e:
            print(f"Error generating detailed reasoning: {e}")
            if 'response' in locals() and hasattr(response, 'text'):
                print(f"API response: {response.text}")
            return ""

In [46]:
import json
import os

time_name_map = {"ccot": "generation_time", "softcot": "total_time", "pause": "generation_time", "icot_si": "generation_time", "codi": "generation_time"}

res_folder = "/home/nee7ne/EfficientCoT/results"
models = ['mistral', 'small']
datasets = ['gsm8k', 'multiarith', 'svamp']
res = {}
for method in os.listdir(res_folder):
    if method == "effi_cot":
        method = "effi_cot/vanilla"
    if not os.path.exists(f"{res_folder}/{method}/{models[0]}/{datasets[0]}") or len(os.listdir(f"{res_folder}/{method}/{models[0]}/{datasets[0]}")) == 0:
        print(f"{res_folder}/{method}/{models[0]}/{datasets[0]}")
        continue
    res[method] = {}
    for m in models:
        res[method][m] = {}
        for d in datasets:
            res[method][m][d] = {}
            if not os.path.exists(f"{res_folder}/{method}/{m}/{d}"):
                print("2")
                continue
            folder_name = [path for path in os.listdir(f"{res_folder}/{method}/{m}/{d}") if os.path.isdir(f"{res_folder}/{method}/{m}/{d}/{path}")]
            if len(folder_name) == 0 or not os.path.exists(f"{res_folder}/{method}/{m}/{d}/{folder_name[0]}/inference_results.json"):
                print("3")
                continue
            with open(f"{res_folder}/{method}/{m}/{d}/{folder_name[0]}/inference_results.json", "r") as f:
                data = json.load(f)
                if len(data) < 10:
                    data = data['results']
            if "sample_time" not in data[0]:
                continue
            res[method][m][d]['time'] = [sample["sample_time"] for sample in data]
            with open(f"{res_folder}/{method}/{m}/{d}/evaluation_results.jsonl", "r") as f:
                lines = f.readlines()
                data = json.loads(lines[-1])
                res[method][m][d]['acc'] = data['summary_acc'] if "summary_acc" in data else data['numerical_accuracy']

/home/nee7ne/EfficientCoT/results/codi/mistral/gsm8k
/home/nee7ne/EfficientCoT/results/pause/mistral/gsm8k
/home/nee7ne/EfficientCoT/results/ccot/mistral/gsm8k


0.12

In [ ]:
for m in sorted(models):
    for d in sorted(datasets):
        acc_line = ""
        time_line = ""
        for method in sorted(res):

            if len(res[method][m][d]) == 0:
                continue
            if isinstance(res[method][m][d]['acc'], str):
                acc_line += f"{float(res[method][m][d]['acc'].split('±')[0]) * 100}\t"
            else:
                acc_line += f"{float(res[method][m][d]['acc']) * 100}\t"
            time_line += f"{np.mean(res[method][m][d]['time']):.2f} ± {np.std(res[method][m][d]['time']):.2f}\t"
        print(acc_line)
        print(time_line)

12.0	15.0	
1.70 ± 0.23	2.30 ± 0.20	
16.0	3.0	
1.42 ± 0.20	2.41 ± 0.04	
42.0	61.0	
1.71 ± 0.27	2.11 ± 0.24	
6.0	8.0	
1.37 ± 0.15	2.24 ± 0.13	
28.000000000000004	3.0	
1.28 ± 0.17	2.49 ± 0.14	
28.999999999999996	51.0	
1.22 ± 0.33	2.00 ± 0.19	


In [ ]:
for m in sorted(models):
    for d in sorted(datasets):
        acc_line = ""
        time_line = ""
        for method in sorted(res):

            if len(res[method][m][d]) == 0:
                continue
            if isinstance(res[method][m][d]['acc'], str):
                acc_line += f"{float(res[method][m][d]['acc'].split('±')[0]) * 100}\t"
            else:
                acc_line += f"{float(res[method][m][d]['acc']) * 100}\t"
            time_line += f"{np.mean(res[method][m][d]['time']):.2f} ± {np.std(res[method][m][d]['time']):.2f}\t"
        print(acc_line)
        print(time_line)

12.0	15.0	
1.70 ± 0.23	2.30 ± 0.20	
3.0	
2.41 ± 0.04	
42.0	61.0	
1.71 ± 0.27	2.11 ± 0.24	
6.0	8.0	
1.37 ± 0.15	2.24 ± 0.13	
28.000000000000004	3.0	
1.28 ± 0.17	2.49 ± 0.14	
28.999999999999996	51.0	
1.22 ± 0.33	2.00 ± 0.19	
